# STEP 1: Installing Required Software

We need to make the following software available on your computer:
* gcloud: access to Google’s cloud services
* kubectl: controlling a Kubernetes cluster
* kustomize: a helper tool that makes it easier to modify kubernetes jobs
* kfctl: controlling Kubeflow specifically

This tutorial is largely based on [Google's end-to-end Kubeflow tutorial](https://www.kubeflow.org/docs/gke/gcp-e2e/).

## Gcloud, Kubectl and kusomize
Unfortunately you cannot install gcloud through Jupyter.  If you are on a Mac you will have to run the following from the command shell to instal gcloud:
```
curl https://sdk.cloud.google.com | bash  # install gcloud
exec -l $SHELL  # restart the shell
gcloud init
```
If you are not on a Mac check out instructions for other systems [here](https://cloud.google.com/sdk/docs/downloads-interactive).

Once you have  gcloud installed you can install kubectl and kustomize like this (again, on a Mac):
```
gcloud components install kubectl
brew install kustomize
```

## kfctl
We don’t “install” kfctl exactly - we just download the executable and put it in a place we can reference.  Download the appropriate version from the [Kubeflow releases page](https://github.com/kubeflow/kubeflow/releases/) into the directory of your choice, unzip it, and then put it on your path.  Example commands (in this case just putting it in the working directory) are:

In [17]:
%%capture
!wget https://github.com/kubeflow/kubeflow/releases/download/v0.5.1/kfctl_v0.5.1_darwin.tar.gz
!tar -xvf kfctl_v0.5.1_darwin.tar.gz

# STEP 2: Creating the Project
Google Cloud divides things into "projects" that can have multiple resources assocaited with them.  If you already have a project you can just use it.  In this section we will create a new project and configure gcloud to point to it.

First use gcloud to tell Google who you are and associate all of this with your google email address.  This command will open up a browser window where you can log in:

In [18]:
%%capture
!gcloud auth application-default login

Then go to the [Google Cloud console page](https://console.cloud.google.com), create a project, and get its ID.  Also **make sure billing is enabled for it**, and choose a geographical region and zone for the project.  Set those decisions as python variables:

In [152]:
PROJECT='kubeflow-245520'
REGION='us-west2'
ZONE='us-west2-c'

Then configure gcloud to point to that project and zone:

In [26]:
!gcloud config set project $PROJECT
!gcloud config set compute/zone $ZONE

Updated property [core/project].
Updated property [compute/zone].


# STEP 3: Create GKE Cluster
Now we will create a Google Kubernetes Engine (GKE) cluster, under the umbrella of the current project, that has kubeflow running on it.

Choose a name for the Kubernetes cluster to use, a name for the GCloud deployment, and login credentials for the web UI of the cluster:

In [1]:
KFAPP='kfapp2'
KUBEFLOW_USERNAME='fcady'
KUBEFLOW_PASSWORD='mypass'

Use kfctl to create the cluster (and a directory of the same name on your local box) and set up its configurations (this will fail if you don’t have billing enabled).  This process will take a while (maybe 20 minutes?) as the resources for the cluster get provisioned.

In [2]:
%env KUBEFLOW_USERNAME=$KUBEFLOW_USERNAME
%env KUBEFLOW_PASSWORD=$KUBEFLOW_PASSWORD
!./kfctl init $KFAPP --platform gcp --project $PROJECT --use_basic_auth -V
%cd $KFAPP
!../kfctl generate all -V --zone $ZONE
!../kfctl apply all -V

env: KUBEFLOW_USERNAME=fcady
env: KUBEFLOW_PASSWORD=mypass
/bin/sh: ./kfctl: No such file or directory
[Errno 2] No such file or directory: 'kfapp2'
/Users/fieldcady/Desktop/model-deployment/kubeflow_integration
/bin/sh: ../kfctl: No such file or directory
/bin/sh: ../kfctl: No such file or directory


After the cluster starts you can see it at ```https://${KFAPP}.endpoints.${PROJECT}.cloud.goog/```

Next connect to Kubeflow cluster+deployment:

In [35]:
#%%capture
!gcloud container clusters get-credentials $KFAPP --zone $ZONE --project $PROJECT

Fetching cluster endpoint and auth data.
kubeconfig entry generated for kfapp2.


# STEP 4: Make Storage Bucket
Now we will set up a cloud storage location where our trained model will be stored:

In [154]:
BUCKET_NAME=PROJECT + '-' + KFAPP + '-bucket'

In [52]:
!echo $ZONE gs://$BUCKET_NAME
!gsutil mb -c regional -l $REGION gs://$BUCKET_NAME

us-west2-c gs://kubeflow-245520-kfapp2-bucket
Creating gs://kubeflow-245520-kfapp2-bucket/...


You should see the bucket listed at ```https://console.cloud.google.com/storage/browser?project=<PROJECT>```.

# STEP 5: Get code, build training Docker image, and Push it to the Registry
The next step for getting Kubeflow running is to get the Docker training image built and stored in GCE where Kubeflow can find it.

This is where you will start changing things for your own projects, but for this walk-through we will use the MNIST example.

In [245]:
import datetime
import time
VERSION_TAG=str(round(time.time()))
TRAIN_IMG_PATH=f"gcr.io/{PROJECT}/{KFAPP}-train:{VERSION_TAG}"
WORKING_DIR="/Users/fieldcady/Desktop/examples/mnist"
print(TRAIN_IMG_PATH)

gcr.io/kubeflow-245520/kfapp2-train:1565117819


In [247]:
%%capture
!git clone https://github.com/kubeflow/examples.git
!docker build -f examples/mnist/Dockerfile.model -t $TRAIN_IMG_PATH examples/mnist
!gcloud auth configure-docker --quiet
!docker push $TRAIN_IMG_PATH

You should then be able to see the image in the Image Registry in gcloud at

```https://console.cloud.google.com/gcr/images/<PROJECT>```

# STEP 6: Create the training job and run it on the cluster

First we cd into the directory with the configuration files for training and use kustomize to make some edits:

In [4]:
%%capture
%cd examples/mnist/training/GCS
!kustomize edit add configmap mnist-map-training --from-literal=secretName=user-gcp-sa
!kustomize edit add configmap mnist-map-training --from-literal=secretMountPath=/var/secrets
!kustomize edit add configmap mnist-map-training --from-literal=GOOGLE_APPLICATION_CREDENTIALS=/var/secrets/user-gcp-sa.json

At this point we **should** be one command away from launching the training job on our Kubeflow cluster.

Alas, Kubeflow is an early-stage open-source project, and that means some rough edges.  In this case there are bugs that improperly handle the example configuration files, and we will need to edit them ourselves.  The two files that need editing are kustomization.yaml in the current directory ("main file") and kustomization.yaml in the base directory ("base file").  The changes that need to be made are:
* In the main file the file Chief_patch.yaml gets added as a path.  But it needs a namespace associated with it that matches the one in the base file.  Do that by adding “namespace: kubeflow” to the properties underneath it.
* The main and base files both have a "vars:" section that defines some properties.  Move all of the properties in the base file to the main file, deleting that section in the base file.

For simplicity, the next cell will write versions of these files with those changes already made:

In [250]:
!git checkout ../base/kustomization.yaml
!git checkout kustomization.yaml

In [257]:
base_file = """apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization

resources:
- Chief.yaml

namespace: kubeflow

generatorOptions:
  disableNameSuffixHash: true

configurations:
- params.yaml
"""

main_file = f"""apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization


configurations:
- params.yaml

# TBD (jinchihe) Need move the image to base file once.
# the issue addressed: kubernetes-sigs/kustomize/issues/1040
# TBD (jinchihe) Need to update the image once
# the issue addressed: kubeflow/testing/issues/373
images:
- name: training-image
  newName: {TRAIN_IMG_PATH}

vars:
- fieldref:
    fieldPath: data.name
  name: trainingName
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.modelDir
  name: modelDir
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.exportDir
  name: exportDir
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.trainSteps
  name: trainSteps
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.batchSize
  name: batchSize
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.learningRate
  name: learningRate
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.GOOGLE_APPLICATION_CREDENTIALS
  name: GOOGLE_APPLICATION_CREDENTIALS
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.secretName
  name: secretName
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training
- fieldref:
    fieldPath: data.secretMountPath
  name: secretMountPath
  objref:
    apiVersion: v1
    kind: ConfigMap
    name: mnist-map-training

patchesJson6902:
- path: Chief_patch.yaml
  target:
    group: kubeflow.org
    kind: TFJob
    name: $(trainingName)
    namespace: kubeflow
    version: v1beta2
resources:
- ../base
configMapGenerator:
- literals:
  - name=mnist-train-dist4
  - trainSteps=20000
  - batchSize=1000
  - learningRate=0.01
  - secretName=user-gcp-sa
  - secretMountPath=/var/secrets
  - GOOGLE_APPLICATION_CREDENTIALS=/var/secrets/user-gcp-sa.json
  - modelDir=gs://{BUCKET_NAME}/
  - exportDir=gs://{BUCKET_NAME}/export
  name: mnist-map-training
"""

_=open('../base/kustomization.yaml', 'w').write(base_file)
_=open('kustomization.yaml', 'w').write(main_file)

Note the name=mnist-train-dist line in the YAML file.  You will have to choose a new name every
time you re-kick-off the training job - otherwise the workflow will not get created.  Now we can kick off the job with this command:

In [259]:
!kustomize build . | kubectl apply -f -

configmap "mnist-map-training-cd2h45k8dm" created
tfjob.kubeflow.org "mnist-train-dist4" created


You can monitor the workflow on the [Google Worklow Page](https://console.cloud.google.com/kubernetes/workload), where you should see a workflow like "mnist-train-dist-chief-0".  **Wait for it to finish.**  After it finishes running then you can copy the exported model file (which is a SavedModel in Tensorflow) to your local machine and zip it up for transfer to Algorithmia.

In [5]:
%%capture
# Copy trained model to local space
!gsutil cp -r gs://$BUCKET_NAME/export .
# Make model/ directory, deleting if it already exists
!rm -rf model*
!mkdir model
# Compress the trained model into a ZIP file
# NOTE: this expects there to be only one model in yoru export/ folder
!cp -r export/$(ls export)/* model/
!zip model.zip -r model

# STEP 7: Loading Model to Algorithmia and Creating an Algorithm

In [6]:
%%capture
# Put zipped model into Algorithmia
!algo rm .my/kubeflow_example force=true
!algo mkdir .my/kubeflow_example
!algo cp model.zip data://.my/kubeflow_example

Then go to [Algorithmia](http://www.algorithmia.com), create a new algorithm (**making sure to give it internet access** for this tutorial), and set its name as a variable:

In [271]:
ALGORITHMIA_USERNAME='fcady'
ALGORITHM_NAME="foo"

FILE_TO_WRITE=ALGORITHM_NAME+'/src/'+ALGORITHM_NAME+'.py'
FILE_TO_COMMIT='src/'+ALGORITHM_NAME+'.py'
REPO_TO_CLONE=ALGORITHMIA_USERNAME+'/'+ALGORITHM_NAME

Now let's use git to checkout this algorithm, write some example code that uses out model, and push it:

In [355]:

!rm -rf $ALGORITHM_NAME  # Delete previously downloaded clone, if it exists
!algo clone $REPO_TO_CLONE

Cloning https://git.algorithmia.com/git/fcady/foo.git
Cloning into 'foo'...
remote: Counting objects: 255, done
remote: Finding sources: 100% (255/255)
remote: Getting sizes: 100% (55/55)
remote: Total 255 (delta 138), reused 255 (delta 138)   
Receiving objects: 100% (255/255), 11.58 MiB | 9.22 MiB/s, done.
Resolving deltas: 100% (138/138), done.
/Users/fieldcady/Desktop/kube/foo
Already up to date.
/Users/fieldcady/Desktop/kube


In [356]:
%%writefile $FILE_TO_WRITE
import Algorithmia
import tensorflow as tf
import requests, zipfile
from os import mkdir, listdir

IMAGE_FNAME  = '/tmp/foo.png'

client = Algorithmia.client()

def extract_model():
    filename = "data://.my/kubeflow_example/model.zip"
    input_zip = client.file(filename).getFile().name
    mkdir("/tmp/unzipped_files")
    zipped_file = zipfile.ZipFile(input_zip)
    return zipped_file.extractall("/tmp/unzipped_files")

def download_image(url):
    with open(IMAGE_FNAME, 'wb') as f:
        f.write(requests.get(url).content)

def create_session(path_to_graph = "/tmp/unzipped_files/model"):
    session = tf.Session()
    tf.saved_model.loader.load(session, ['serve'], path_to_graph)
    y = session.graph.get_tensor_by_name('Softmax:0')
    x = session.graph.get_tensor_by_name('Placeholder:0')
    return (y, x, session)

extract_model()
Y, X, SESSION = create_session()

def classify_image():
    img = tf.keras.preprocessing.image.load_img(IMAGE_FNAME).resize((28,28))
    x = tf.keras.preprocessing.image.img_to_array(img)
    xx = x.mean(axis=2).reshape((1,28,28)) / 255
    predict_values = tf.argmax(Y, 1)
    ret = predict_values.eval(session=SESSION,feed_dict={X: xx})
    return int(ret)

def apply(input):
    try:
        download_image(input['url'])
        msg = 'downloaded image'
    except Exception as e:
        msg = 'failed to get image:' + str(e)
    try: label = classify_image()
    except Exception as e:
        label = str(e)
    output = {
        'label': label,
        'msg': msg
    }
    return output

Overwriting foo/src/foo.py


In [357]:
%%capture
%cd $ALGORITHM_NAME
!git commit -a -m "Code for the algorithm, from Jupyter"
%cd ..
%cd $ALGORITHM_NAME
!git commit -a -m "Code for the algorithm, from Jupyter"
%cd ..

/Users/fieldcady/Desktop/kube/foo
[master e45bdd3] Code for the algorithm, from Jupyter
 Committer: field-cady <fieldcady@Fields-MacBook-Pro.local>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 4 insertions(+), 1 deletion(-)
/Users/fieldcady/Desktop/kube
/Users/fieldcady/Desktop/kube/foo
On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
/Users/fieldcady/Desktop/kube


In [359]:
# Occassionally you have to run this twice...
%cd $ALGORITHM_NAME
!git push
%cd ..

/Users/fieldcady/Desktop/kube/foo
Everything up-to-date
/Users/fieldcady/Desktop/kube


Now let's build the algorithm!

In [360]:
import Algorithmia, os
api_key = os.environ['ALGORITHMIA_API_KEY']
client = Algorithmia.client(api_key)
client.algo(algo_namespace).compile()
client.algo(algo_namespace).publish()

{'compilation': {'output': None, 'successful': True},
 'details': {'label': 'foo', 'summary': None, 'tagline': 'bar'},
 'name': 'foo',
 'resource_type': 'algorithm',
 'self_link': None,
 'settings': {'algorithm_callability': 'private',
              'environment': None,
              'language': None,
              'license': 'apl',
              'network_access': 'full',
              'package_set': 'tensorflow-gpu-1.12',
              'pipeline_enabled': False,
              'royalty_microcredits': 0,
              'source_visibility': 'open'},
 'version_info': {'git_hash': 'dcf13b8bfe2b363f1ab9281d76ce81ad0bd7f2a4',
                  'release_notes': None,
                  'sample_input': None,
                  'sample_output': None,
                  'semantic_version': '1.0.6'}}

In [363]:
algo_namespace = "{}/{}".format(ALGORITHMIA_USERNAME, ALGORITHM_NAME)
latest_hash = client.algo(algo_namespace).info().version_info.git_hash
algo_input = {
    #"url": "https://edwin-de-jong.github.io/blog/mnist-sequence-data/fig/5.png"
    "url": "https://miro.medium.com/max/490/1*nlfLUgHUEj5vW7WVJpxY-g.png"
}
res = client.algo(algo_namespace+'/'+latest_hash).pipe(algo_input).result
print(res)

{'/tmp/': ['tmp5b0b4olo', 'algoout', 'unzipped_files', '.X11-unix', 'foo.png'], '/tmp/unzipped_files/model/': ['variables', 'saved_model.pb'], 'label': 0, 'msg': 'downloaded_fine', 'status': 'extracted,created,downloaded,labelled'}
